### 3. Use Gibbs Sampling

In [2]:
import numpy as np
from itertools import product

# Function to calculate the sum of neighbors for a given node in the lattice
def calculate_neighbors_sum(lattice, i, j, n, use_periodic_boundary):
    if use_periodic_boundary:
        # If periodic boundary conditions are used, the grid is wrapped around like a torus
        return (
            lattice[(i+1)%n, j] +  # Down neighbor
            lattice[i, (j+1)%n] +  # Right neighbor
            lattice[(i-1)%n, j] +  # Up neighbor
            lattice[i, (j-1)%n]    # Left neighbor
        )
    else:
        # If fixed boundary conditions are used, edges and corners have fewer neighbors
        neighbors = 0
        if i > 0: neighbors += lattice[i-1, j]  # Neighbor above, if not on top edge
        if i < n-1: neighbors += lattice[i+1, j]  # Neighbor below, if not on bottom edge
        if j > 0: neighbors += lattice[i, j-1]  # Neighbor to the left, if not on left edge
        if j < n-1: neighbors += lattice[i, j+1]  # Neighbor to the right, if not on right edge
        return neighbors

# Function to calculate the conditional probability of a spin being up or down
def conditional_probability(beta, s, neighbors_sum, num_neighbors):
    # Calculate the normalized sum of neighboring spins
    normalized_sum = neighbors_sum / num_neighbors
    # Calculate the conditional probability using the Boltzmann distribution
    return np.exp(beta * s * normalized_sum) / (
        np.exp(beta * normalized_sum) + np.exp(-beta * normalized_sum)
    )

# Function to perform Gibbs sampling over the lattice
def gibbs_sampling(lattice, beta, n, iterations, use_periodic_boundary):
    # Initialize a matrix to store the joint probabilities of the lattice
    joint_probabilities = np.zeros((2, 2))
    for _ in range(iterations):
        # Iterate over each node in the lattice
        for i, j in product(range(n), repeat=2):
            # Assume 4 neighbors for nodes not on edges or corners
            num_neighbors = 4
            # Calculate sum of neighbors for the current node
            neighbors_sum = calculate_neighbors_sum(lattice, i, j, n, use_periodic_boundary)
            # Calculate conditional probability for the node to be in the up state
            p_plus = conditional_probability(beta, +1, neighbors_sum, num_neighbors)
            # Update the node's state based on the conditional probability
            lattice[i, j] = 1 if np.random.rand() < p_plus else -1

        # After a burn-in period, start accumulating joint probabilities. The first hald of iterations were used as a burn-in period.            
        if _ > iterations // 2:
            # Increment the count for the current state of nodes x_{1,10} and x_{10,10}
            joint_probabilities[(lattice[1, n-1] + 1) // 2, (lattice[n-1, n-1] + 1) // 2] += 1

    # Normalize the joint probabilities
    joint_probabilities /= joint_probabilities.sum()
    return joint_probabilities

# Simulation parameters
n = 10  # Size of the lattice (10x10)
beta_values = [4, 3.5, 3, 2, 1.5, 1, 0.7, 0.5, 0.1, 0.01]  # List of beta values to simulate
iterations = 5000000  # Number of iterations for the Gibbs sampling
use_periodic_boundary = False  # Flag to determine boundary condition type

# Perform Gibbs sampling for each beta value
for beta in beta_values:
    # Initialize the lattice with random spins
    initial_lattice = np.random.choice([-1, 1], size=(n, n))
    # Perform Gibbs sampling to calculate joint probabilities
    joint_probabilities = gibbs_sampling(initial_lattice, beta, n, iterations, use_periodic_boundary)
    # Print out the joint probabilities for the current beta value
    print(f"Joint probabilities for beta = {beta}:\n{joint_probabilities}\n")
